In [1]:
import pandas as pd
import xlrd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df1 = pd.read_excel('Central Branch  trackers/Central Division 2021 Debris Removal Tracker.xlsx', parse_dates=True)
df2 = pd.read_csv('../tetra forms ASB and SA/Site Assessment Tracker.csv', parse_dates=True)

In [3]:
df3 =pd.read_excel("../tetra forms ASB and SA/SA_TetraForms1.xlsx", parse_dates=True)

In [4]:
smartsheets = df1.copy()
sa = df2.copy()
tetarforms = df3.copy()

In [5]:
# smartsheets[(smartsheets['Site Assessment'].notnull()) & (smartsheets['RAD_MERC Sweep'].isnull())].to_excel('No sweep dates.xlsx')

In [6]:
# smartsheets.info(verbose=True)

# Site Assessment site up

In [7]:
# sa column selection
sa = sa[['APN','Date Completed','Automobiles', 'Motorcycle', 'ATV/UTV/Snowmobiles',
       'Ag/Commercial/Construction', 'RV', 'Trailers', 'Vessel', 'Chimney','Division']]

KeyError: "['ATV/UTV/Snowmobiles', 'Division', 'Vessel', 'Date Completed', 'Chimney', 'RV', 'Trailers', 'Motorcycle', 'Ag/Commercial/Construction', 'Automobiles'] not in index"

In [ ]:
# only use central division APNs and parcel information
sa = sa[sa['Division'].isin(['Central Division'])]

### Get the value counts of APNS in Sa

In [ ]:
pd.set_option('display.max_rows',1000)

In [ ]:
sa['APN'].value_counts().to_excel('SA APN counts central.xlsx')

# other Setups

In [ ]:
# replace all 0s to No and all 1 to Yes for Chimney to match smartsheets
sa['Chimney'] = sa['Chimney'].fillna(0).astype(int).astype(str).str.replace("0", "No").str.replace("1", "Yes")


In [ ]:
# set up date columns
sa['Date Completed'] = pd.to_datetime(sa['Date Completed']).dt.date

In [ ]:
sa.rename(columns={'Date Completed': 'SA Survey Date','APN': 'APN_SA','Motorcycle':'Motorcycle_SA',
                  'RV': 'RV_SA', 'Trailers':'Trailers_SA', 'Vessel': 'Vessel_SA'}, inplace=True)

In [ ]:
# set up numerical values for al vehicles
sa['Automobiles'] = pd.to_numeric(sa['Automobiles'])
sa['Motorcycle_SA'] = pd.to_numeric(sa['Motorcycle_SA'])
sa['ATV/UTV/Snowmobiles'] = pd.to_numeric(sa['ATV/UTV/Snowmobiles'])
sa['Ag/Commercial/Construction'] = pd.to_numeric(sa['Ag/Commercial/Construction'])
sa['RV_SA'] = pd.to_numeric(sa['RV_SA'])
sa['Trailers_SA'] = pd.to_numeric(sa['Trailers_SA'])
sa['Vessel_SA'] = pd.to_numeric(sa['Vessel_SA'])

In [ ]:
sa.fillna({'SA Survey Date':0,
           'Automobiles': 0,
           'Motorcycle_SA': 0,
           'ATV/UTV/Snowmobiles': 0,
           'Ag/Commercial/Construction': 0,
           'RV_SA': 0,
           'Trailers_SA': 0,
           'Vessel_SA': 0}, inplace=True)

In [ ]:
sa.columns = map(str.upper, sa.columns)

# Tetra Forms setup

In [ ]:
# get just the counties we need
tetarforms = tetarforms[tetarforms['county'].isin(['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER'])]

### Get the value counts of APNS in Tetra Forms for central

In [ ]:
tetarforms['apn'].value_counts().to_excel('Tetra Forms APN counts SA central.xlsx')

In [ ]:
# format dates 
tetarforms['survey_date'] = pd.to_datetime(tetarforms['survey_date']).dt.date

In [ ]:
#get columsn we need
tetarforms = tetarforms[['apn','survey_date','chimney_quantity', 'automobile_quantity', 'motorcycle_quantity',
                         'atv_utv_quantity', 'comm_equip_quantity', 'rvs_quantity', 'trailer_quantity', 'vessel_quantity']]

In [ ]:
tetarforms.fillna({'survey_date':0,
           'automobile_quantity': 0,
           'motorcycle_quantity': 0,
           'atv_utv_quantity': 0,
           'comm_equip_quantity': 0,
           'rvs_quantity': 0,
           'trailer_quantity': 0,
           'vessel_quantity': 0}, inplace=True)

In [ ]:
tetarforms['chimney_quantity'] = np.where(tetarforms['chimney_quantity'] > 0, 'Yes', 'No')

In [ ]:
tetarforms.rename(columns={'apn': 'APN_TF', 'motorcycle_quantity':'motorcycle_quantity_TF'}, inplace=True)

In [ ]:
tetarforms.columns = map(str.upper, tetarforms.columns)

In [ ]:
# merge1 tetraforms and SA
merge1 = sa.merge(tetarforms,
                 left_on='APN_SA',
                 right_on='APN_TF',
                 how='outer',
                 suffixes=('_SA', '_TF'))

# Smart sheet set up

In [ ]:
# smartsheets.info(verbose=True, show_counts=True)

In [ ]:
# smart sheet column selection
smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status','County',
                           'Chimney',
                           'Site Assessment',
                          'Number of Passenger Vehicles', 'Number of ATV_UTV_snowmobiles',
                          'Number of Ag_Commercial_Construction Equipment','Number of Motorcycles',
                          'Number of RV\'s', 'Number of Trailers', 'Number of Vessels (Boat_Jetski)', 'Total Number of Vehicles']]

In [ ]:
smartsheets = smartsheets.copy()

In [ ]:
# take of the time on the dates
smartsheets['Site Assessment'] = smartsheets['Site Assessment'].dt.date

In [ ]:
smartsheets.fillna({'Chimney': 'No',
                    'Site Assessment': 0,
                    'Number of Passenger Vehicles': 0,
                    'Number of ATV_UTV_snowmobiles': 0,
                    'Number of Ag_Commercial_Construction Equipment': 0,
                    'Number of Motorcycles': 0,
                    'Number of RV\'s':0,
                    'Number of Trailers': 0,
                    'Number of Vessels (Boat_Jetski)': 0}, inplace=True)

In [ ]:
smartsheets.rename(columns={'County':'County_SS','Site Assessment': 'SA Survey Date', 'Number of Passenger Vehicles': 'Automobiles',
                           'Number of ATV_UTV_snowmobiles': 'ATV/UTV/Snowmobiles',
                           'Number of Ag_Commercial_Construction Equipment': 'Ag/Commercial/Construction',
                           'Number of Motorcycles': 'Motorcycles_SS', 'Number of RV\'s': 'RV_SS',
                           'Number of Trailers':'Trailers_SS','Number of Vessels (Boat_Jetski)':'Vessel_SS'}, inplace=True)

In [ ]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [ ]:
# merge both data frames for QC
final = smartsheets.merge(merge1,
                         left_on='APN_ROW SEGMENT',
                         right_on='APN_SA',
                         how='outer',
                         suffixes=('_SS', '_SA'))

In [ ]:
# column orinizationa
final = final[['APN_ROW SEGMENT','APN_SA', 'APN_TF','STREET #','STREET NAME','STRUCTURAL STATUS','COUNTY_SS',
               'SA SURVEY DATE_SS', 'SA SURVEY DATE_SA','SURVEY_DATE',
               'CHIMNEY_SS','CHIMNEY_SA','CHIMNEY_QUANTITY',
               'AUTOMOBILES_SS', 'AUTOMOBILES_SA','AUTOMOBILE_QUANTITY',
               'MOTORCYCLES_SS', 'MOTORCYCLE_SA', 'MOTORCYCLE_QUANTITY_TF',
              'ATV/UTV/SNOWMOBILES_SS', 'ATV/UTV/SNOWMOBILES_SA','ATV_UTV_QUANTITY',
              'AG/COMMERCIAL/CONSTRUCTION_SS', 'AG/COMMERCIAL/CONSTRUCTION_SA','COMM_EQUIP_QUANTITY',
              'RV_SS', 'RV_SA','RVS_QUANTITY',
              'TRAILERS_SS', 'TRAILERS_SA','TRAILER_QUANTITY',
              'VESSEL_SS','VESSEL_SA','VESSEL_QUANTITY',
               'DIVISION','TOTAL NUMBER OF VEHICLES']]

# compare function for each category

In [ ]:
def compare_cols(df,comp1,comp2,comp3):
    if (df[comp1] == df[comp2]) & (df[comp1] == df[comp3]):
        return 1
    else:
        return 0

# Survey date checks

In [ ]:
final.insert(final.columns.get_loc('SURVEY_DATE')+1, "Survey Matches",
            final.apply(compare_cols, comp1='SA SURVEY DATE_SS', comp2='SA SURVEY DATE_SA', comp3='SURVEY_DATE', axis=1))

# CHIMNEY

In [ ]:
final.insert(final.columns.get_loc('CHIMNEY_QUANTITY')+1, "Chimney Macthes",
            final.apply(compare_cols, comp1='CHIMNEY_SS', comp2='CHIMNEY_SA', comp3='CHIMNEY_QUANTITY', axis=1))

# AUTOMOBILES

In [ ]:
final.insert(final.columns.get_loc('AUTOMOBILE_QUANTITY')+1, "AUTOMOBILE-M",
            final.apply(compare_cols, comp1='AUTOMOBILES_SS', comp2='AUTOMOBILES_SA', comp3='AUTOMOBILE_QUANTITY', axis=1))

# MOTORCYCLES

In [ ]:
final.insert(final.columns.get_loc('MOTORCYCLE_QUANTITY_TF')+1, "Motorcycle-M",
            final.apply(compare_cols, comp1='MOTORCYCLES_SS', comp2='MOTORCYCLE_SA', comp3='MOTORCYCLE_QUANTITY_TF', axis=1))

# ATV/UTV/SNOWMOBILES

In [ ]:
final.insert(final.columns.get_loc('ATV_UTV_QUANTITY')+1, "ATV-M",
            final.apply(compare_cols,
                        comp1='ATV/UTV/SNOWMOBILES_SS', comp2='ATV/UTV/SNOWMOBILES_SA', comp3='ATV_UTV_QUANTITY', axis=1))

# AG/COMMERCIAL/CONSTRUCTION

In [ ]:
final.insert(final.columns.get_loc('COMM_EQUIP_QUANTITY')+1, "COMM-M",
            final.apply(compare_cols,
                        comp1='AG/COMMERCIAL/CONSTRUCTION_SS',
                        comp2='AG/COMMERCIAL/CONSTRUCTION_SA', comp3='COMM_EQUIP_QUANTITY', axis=1))

# RV

In [ ]:
final.insert(final.columns.get_loc('RVS_QUANTITY')+1, "RV-M",
            final.apply(compare_cols, comp1='RV_SS', comp2='RV_SA', comp3='RVS_QUANTITY', axis=1))

# TRAILERS

In [ ]:
final.insert(final.columns.get_loc('TRAILER_QUANTITY')+1, "Trailer-M",
            final.apply(compare_cols, comp1='TRAILERS_SS', comp2='TRAILERS_SA', comp3='TRAILER_QUANTITY', axis=1))

# VESSEL

In [ ]:
final.insert(final.columns.get_loc('VESSEL_QUANTITY')+1, "Vessel-M",
            final.apply(compare_cols, comp1='VESSEL_SS', comp2='VESSEL_SA', comp3='VESSEL_QUANTITY', axis=1))

# color in the compare columns

In [ ]:
final_style = final.style.set_properties(**{'background-color': 'lightblue'},
                          subset=['Survey Matches', 'Chimney Macthes','AUTOMOBILE-M','Motorcycle-M',"ATV-M",
                                  'COMM-M',"RV-M",'Trailer-M' ,'Vessel-M'])

In [ ]:
final_style.to_excel('Central Dixie Fire SA Audit.xlsx')